In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
forest = RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [2]:
train = pd.read_csv('../data/train_ver04.csv', index_col='STU_ID')
train.head()

,BYSTUWT,BYSEX,BYRACE,BYSTLANG,BYHOMLNG,BYSTLNG2,BYDOB_P,BYFCOMP,BYPARED,BYMOTHED,...,BYS24D,BYS24E,BYS28,BYS71A,BYS71B,BYS71C,BYS71D,BYS71E,BYS71F,F3ONET2CURR
STU_ID,,,,,,,,,,,,,,,,,,,,,
101102,28.2951,2.0,2.0,0.0,4.0,2,198605.0,1.0,5.0,5.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,29
101104,589.7248,2.0,7.0,1.0,1.0,4,198601.0,1.0,2.0,2.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,27
101105,235.7822,2.0,3.0,1.0,1.0,4,198607.0,5.0,2.0,2.0,...,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,39
101106,178.9513,2.0,4.0,0.0,2.0,2,198511.0,1.0,1.0,1.0,...,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,29
101108,192.4304,1.0,4.0,1.0,1.0,4,198607.0,1.0,6.0,6.0,...,2.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,53


In [3]:
y = train.F3ONET2CURR
x = train.drop(columns='F3ONET2CURR')

In [6]:
forest.fit(x,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [7]:
imp = forest.feature_importances_
idx = np.argsort(imp)[::-1]
labels = x.columns
for i in range(len(labels)):
    print(labels[i], imp[idx[i]])

BYSTUWT 0.016585803818228606
BYSEX 0.014890365517149708
BYRACE 0.014442835093745571
BYSTLANG 0.014422935142362814
BYHOMLNG 0.014178278335848666
BYSTLNG2 0.014145463134718546
BYDOB_P 0.014116175740754336
BYFCOMP 0.014079747714187825
BYPARED 0.014066978926729033
BYMOTHED 0.014050417344360315
BYFATHED 0.01403218061097037
BYOCCUM 0.013971750217892103
BYOCCUF 0.013689271447721287
BYINCOME 0.013191909205970645
BYSES1QU 0.013058937322039037
BYSES2QU 0.012998979711634509
BYSCHPRG 0.012774738260863153
BYSQSTAT 0.012627060957509645
BYTXSTAT 0.012464501652115045
BYPQSTAT 0.012412955018627164
BYTXCSTD 0.012244174871855711
BYNELS2M 0.01208598529786669
BYNELS2R 0.012064168636587587
BYPISAME 0.011792279245149246
BYPISARE 0.01149363722922327
BYTXMIRR 0.011364445602515066
BYTXMSTD 0.011344040149175737
BYTX1MPP 0.011200590155633615
BYTX2MPP 0.010599908964707382
BYTX3MPP 0.010391579913073027
BYTX4MPP 0.010263530102389257
BYTX5MPP 0.009851816781331665
BYTXRIRR 0.009707634540410648
BYTXRSTD 0.0097016422862

In [10]:
x_filter = x[labels[imp > 0.01]]
x_filter.shape

(9955, 31)

In [11]:
x.shape

(9955, 151)

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [14]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler

In [33]:
pipe = Pipeline([('전처리',StandardScaler()), ('알고리즘', KNeighborsClassifier())])

In [34]:
# param = [
#     {'알고리즘' : [KNeighborsClassifier()],
#      '전처리' : [StandardScaler(), MinMaxScaler(),None],
#      '알고리즘_n_neighbors' : np.arange(1,100),
#      '알고리즘_weights' : ['uniform', 'distance'],
#      '알고리즘_p' : [1,2]},
    
#     {'알고리즘' : [RandomForestClassifier()],
#      '전처리' : [StandardScaler(), MinMaxScaler(),None],
#      '알고리즘_n_neighbors' : [300],
#      '알고리즘_criterion' : ['gini', 'entropy'],
#      '알고리즘_max_depth' : [None] + list(np.arange(5,100))}]

param = [
    {'알고리즘':[KNeighborsClassifier()],
     '전처리' : [StandardScaler()],
     '알고리즘__n_neighbors': np.arange(1,100),
     '알고리즘__weights': ['uniform','distance'],
     '알고리즘__p':[1,2]},
    {'알고리즘':[RandomForestClassifier()],
     '전처리': [MinMaxScaler(), None],
     '알고리즘__n_estimators': [300],
     '알고리즘__criterion': ['gini','entropy'],
     '알고리즘__max_depth': [None]+list(np.arange(5,100))}]

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
grid_full = GridSearchCV(estimator=pipe, param_grid=param,
                          scoring='accuracy', cv=4, n_jobs=-1, verbose=1)

In [37]:
grid_full.fit(x_filter, y)

Fitting 4 folds for each of 780 candidates, totalling 3120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 35.0min
[Parallel(n_jobs=-1)]: Done 3120 out of 3120 | elapsed: 91.1min finished


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('전처리', StandardScaler()),
                                       ('알고리즘', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid=[{'알고리즘': [KNeighborsClassifier()],
                          '알고리즘__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58,...
                          '알고리즘__weights': ['uniform', 'distance'],
                          '전처리': [StandardScaler()]},
                         {'알고리즘': [RandomForestClassifier(max_depth=10,
                                                          n_estimators=300)],
                          '알고리즘__criterion': ['gini', 'entropy'],
                          '알고리즘__max_depth': [None, 5, 6, 7, 8, 9, 10, 11, 12,
                             

In [38]:
grid_full.best_score_

0.2342551196324923

In [40]:
grid_full.best_estimator_

Pipeline(steps=[('전처리', None),
                ('알고리즘',
                 RandomForestClassifier(max_depth=10, n_estimators=300))])

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y)

In [6]:
y_train.value_counts(normalize=True)

43    0.145058
41    0.088669
11    0.086526
29    0.073399
25    0.071256
35    0.068444
13    0.058398
51    0.048219
39    0.046209
31    0.041789
53    0.036030
27    0.034557
47    0.032949
49    0.032146
15    0.029467
33    0.025717
21    0.023172
17    0.020493
19    0.018886
37    0.018618
Name: F3ONET2CURR, dtype: float64

In [7]:
y_test.value_counts(normalize=True)

43    0.145038
41    0.088791
11    0.086782
29    0.073524
25    0.071113
35    0.068702
13    0.058658
51    0.048212
39    0.046203
31    0.041784
53    0.035757
27    0.034552
47    0.032945
49    0.032141
15    0.029329
33    0.025713
21    0.022901
17    0.020490
19    0.018883
37    0.018481
Name: F3ONET2CURR, dtype: float64

In [8]:
import joblib
with open('../data/my_val.pkl3', 'wb') as f:
    joblib.dump({'x':x_train,
                 'y':y_train,
                 'val_x':x_test,
                 'val_y':y_test},f)

## 2. Baseline model tunning

In [9]:
import joblib

import pandas as pd
import numpy as np


from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [10]:
with open('../data/my_val.pkl3', 'rb') as f:
    data = joblib.load(f)
locals().update(data)
del data

In [11]:
test = pd.read_csv('../data/test_ver04.csv', index_col='STU_ID')
test = test.drop(columns='F3ONET2CURR')
test.head()

,BYSTUWT,BYSEX,BYRACE,BYSTLANG,BYHOMLNG,BYSTLNG2,BYDOB_P,BYFCOMP,BYPARED,BYMOTHED,...,BYS24C,BYS24D,BYS24E,BYS28,BYS71A,BYS71B,BYS71C,BYS71D,BYS71E,BYS71F
STU_ID,,,,,,,,,,,,,,,,,,,,,
310221,575.7933,2.0,6.0,1.0,1.0,4,198412.0,2.0,4.0,4.0,...,4.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
228218,281.6741,1.0,7.0,1.0,1.0,4,198507.0,6.0,1.0,1.0,...,3.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
179231,275.0641,1.0,7.0,1.0,1.0,4,198607.0,1.0,4.0,4.0,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
215101,149.2901,2.0,3.0,1.0,1.0,4,198507.0,2.0,3.0,1.0,...,4.0,1.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
245121,186.5067,1.0,7.0,1.0,1.0,4,198601.0,5.0,3.0,3.0,...,2.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
# base_model = RandomForestClassifier(max_depth=10, n_estimators=300)
pipe = Pipeline([('전처리',MinMaxScaler()), 
                ('알고리즘', RandomForestClassifier(max_depth=20, n_estimators=300))])    

# pipe = Pipeline([('전처리',MinMaxScaler()), 
#                 ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=1000, criterion='entropy', max_features='auto', min_samples_split=2,n_jobs=-1))])   

In [21]:
# base_model.fit(x, y)
pipe.fit(x, y)

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=20, n_estimators=300))])

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, pipe.predict(val_x))

0.2531137002812374

In [23]:
pred = pipe.predict(test)
pred

array([43, 51, 43, ..., 43, 43, 15], dtype=int64)

In [24]:
pred.shape

(2479,)

In [25]:
test.shape

(2479, 151)

In [26]:
submission = pd.DataFrame(pred, index=test.index, columns=['PRED']).reset_index()
submission

,STU_ID,PRED
0,310221,43
1,228218,51
2,179231,43
3,215101,41
4,245121,51
...,...,...
2474,439214,11
2475,106206,13
2476,272131,43
2477,350228,43


In [27]:
submission.to_csv('my_pred36.csv', index=False)